In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import bokeh
import seaborn
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
import pandas as pd


In [2]:
def create_scurve_result(df):
    row_cnt = 0
    columns = ['DayNumber','Conv_Total','Conv']
    df_new = pd.DataFrame(columns=columns)
    total_rows = len(df.index)
    for i in range (0,459):
        found = df.loc[(df['conv_gap'] == i)]
        row_cnt = (len(found.index) + row_cnt) 
        if total_rows > 0:
            conv_running = (row_cnt / total_rows) * 100
        else:
            conv_running=0
        df_new.loc[i] = [i,conv_running,(len(found.index))]
    return df_new

In [3]:
def create_scurve_graph(DF1_RR,DF2_RR,L1,L2,ymax,xmax,trial_length):
    X1=DF1_RR['X'].values
    Y1=DF1_RR['Y'].values

    X2=DF2_RR['X'].values
    Y2=DF2_RR['Y'].values

    if trial_length == '3':
        shutoff = 90
        otm1 = 7
        otm2 = 62
        otm3 = 97
        otm4 = 130
        #xmax = 100
    
    if trial_length == '6':
        shutoff = 180
        otm1 = 7
        otm2 = 152
        otm3 = 187
        otm4 = 220
        xmax = 270        
    
    if trial_length == '12':
        shutoff = 365
        otm1 = 7
        otm2 = 337
        otm3 = 372
        otm4 = 405
        xmax = 455        

    
    fig = plt.figure(figsize=(15, 20),frameon =True)
      
    ax1 = plt.subplot(211)    
    ax1.set_title(" ",fontsize=15)
    ax1.plot(X1,Y1,label=L1)
    ax1.plot(X2,Y2,label=L2)

    ax1.set_xlabel('Days from TrialActivation',fontsize=15)
    ax1.set_ylabel('Cumulative Conversion %',fontsize=15)
    ax1.set_ylim(0,ymax) 
    ax1.set_xlim(-1,xmax) 
    ax1.grid(True, linestyle='-.')
    
    ax1.set_yticks(np.arange(0, ymax, 0.5))
    ax1.set_xticks(np.arange(0, xmax, 5))
    ax1.tick_params(direction='out', length=6, width=2, colors='k', grid_color='k', grid_alpha=0.5)
    ax1.legend(loc='best', ncol=2, shadow=True, fancybox=False)
    
    #ax1.axes.get_yaxis().set_visible(False)
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.spines['left'].set_visible(False)
    ax1.get_xaxis().tick_bottom()
    ax1.get_yaxis().tick_left()
    
    filename=L1+".png"
    fig.savefig(filename,transparent=False,  bbox_inches='tight',pad_inches=0.15,dpi=200)   
    plt.show()
    
    return filename

In [4]:
Y=60
X=6

# IsConversion, activationdate,conversiondate,            
        
#DF1 = FINAL_DF1
#DF1_R = create_scurve_result(DF1)
#DF1_R = DF1_R.drop(columns=['Conv'])
#DF1_R = DF1_R.rename(index=str,columns={"DayNumber":"X","Conv_Total":"Y"})

#DF2 = FINAL_DF2
#DF2_R = create_scurve_result(DF2)
#DF2_R = DF2_R.drop(columns=['Conv'])
#DF2_R = DF2_R.rename(index=str,columns={"DayNumber":"X","Conv_Total":"Y"})

#print("T",len(FINAL_DF2))
#print("C",len(FINAL_DF1))

#create_scurve_graph(DF1_R,DF2_R,"Control","Test",X,Y,'3')